In [40]:
import json
import nltk
import operator
from nltk.corpus import stopwords as sw

directory='/home/patrosau@alumno.upv.es/Descargas/hispatweets/'

def getFileName(directory, country, userId):
    return directory + country + '/' + user + '.json'

In [35]:
from nltk.tokenize import TweetTokenizer

def tokenizeTweet(tweet):

    # eliminamos mencion de usuarios, quitamos las repeticiones de mas de 3 letras y pasamos a minusculas
    tokenizer = TweetTokenizer(strip_handles=True, reduce_len=True, preserve_case=False)
    
    return tokenizer.tokenize(tweet)

In [19]:
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [8]:
spanish_tagged_words = nltk.corpus.conll2002.tagged_words()

In [9]:
for tagged_words in spanish_tagged_words[:10]:
    print (tagged_words)

('Sao', 'NC')
('Paulo', 'VMI')
('(', 'Fpa')
('Brasil', 'NC')
(')', 'Fpt')
(',', 'Fc')
('23', 'Z')
('may', 'NC')
('(', 'Fpa')
('EFECOM', 'NP')


In [22]:
from nltk.corpus import cess_esp
corpus_sentences=cess_esp.tagged_sents()

words_tagged = dict()
for sentence in corpus_sentences:
    for token_tag in sentence:
        if (token_tag[0] not in words_tagged):
            words_tagged[token_tag[0]] = token_tag[1]

In [31]:
pronombres = ['mia', 'mio', 'yo', 'mi', 'me', 'tu', 'te', 'ti', 'contigo', 'usted', 'él', 'ella', 'consigo', 'nosotros','nosotras', 'nos', 'vosotros', 'vosotras', 'os', 'vos', 'ellos', 'ellas',
             'mío','tuyo' ,'suyo','nuestro','vuestro','suyo', 'míos' ,'tuyos','suyos','nuestros','vuestros','suyos', 'mía' ,'tuya' ,'suya','nuestra','vuestra','suya',
             'mías','tuyas','suyas' ,'nuestras' ,'vuestras','suyas']

In [27]:
def getTag(word):
    if word in words_tagged:
        return words_tagged[word]
    else:
        if word in pronombres:
            return 'p'
    return '-'

In [32]:
text = "la casa es bonita como la mia y la tuya el este yo vale esta esa vuestra"
tokens = text.split(' ');
for token in tokens:
    print (token, getTag(token))

la da0fs0
casa ncfs000
es vsip3s0
bonita aq0fs0
como cs
la da0fs0
mia p
y cc
la da0fs0
tuya p
el da0ms0
este dd0ms0
yo pp1csn00
vale vmip3s0
esta dd0fs0
esa dd0fs0
vuestra p


In [45]:
# calcula las caracteristicas de un autor
# numero de determinantes / normalizado por numero total palabras
# numero de adjetivos / normalizado por numero total de palabras
# numero de ''de'' / normalizado por numero total de palabras
# numero de pronombres / normalizado por numero total de palabras
# numero de 'para' / normalizado por numero total de palabras
# numero de 'con' / normalizado por numero total de palabras
# numero de 'no' / normalizado por numero total de palabras
# numero de verbos presente / normalizado por numero total de palabras
def getAuthorFeatures(fileName):

    wordCount = 0
    detCount = 0
    adjCount = 0
    deCount = 0
    pronCount = 0
    paraCount = 0
    conCount = 0
    vbpCount = 0
    noCount = 0

    
    with open(fileName) as f:
        for line in f:
            tokens = tokenizeTweet(json.loads(line)['text'])
            for token in tokens:

                wordCount = wordCount + 1
                if token == 'de':
                    deCount = deCount + 1
                elif token == 'con':
                    conCount = conCount + 1
                elif token == 'para':
                    paraCount = paraCount + 1
                elif token == 'no':
                    noCount = noCount + 1
                else:
                    tag = getTag(token)
                    if tag.startswith('v'):
                        vbpCount = vbpCount + 1
                    if tag.startswith('d'):
                        detCount = detCount + 1
                    if tag.startswith('a'):
                        adjCount = adjCount + 1
                    if tag.startswith('p'):
                        pronCount = pronCount + 1

    if wordCount > 0:
        return [detCount/wordCount, adjCount/wordCount, deCount/wordCount, pronCount/wordCount, paraCount/wordCount, conCount/wordCount, vbpCount/wordCount, noCount/wordCount]
    else:
        return [0, 0, 0, 0, 0, 0, 0, 0]

In [48]:
with open(directory + 'sextrainingdata.txt', 'w') as train:
    with open(directory + 'training.txt') as trainingFile:
        for line in trainingFile:
            [user, country, sex] = line.split(':::')
            train.write(user + ',')
            userVector = getAuthorFeatures(getFileName(directory, country, user))
            for feature in userVector:
                train.write(str(feature) + ',')
            train.write(sex + '\n')

In [49]:
with open(directory + 'sextestdata.txt', 'w') as train:
    with open(directory + 'test.txt') as trainingFile:
        for line in trainingFile:
            [user, country, sex] = line.split(':::')
            train.write(user + ',')
            userVector = getAuthorFeatures(getFileName(directory, country, user))
            for feature in userVector:
                train.write(str(feature) + ',')
            train.write(sex + '\n')